Install required packages

In [ ]:
# %pip install requests librosa matplotlib

import requests
import time
import json
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Write a function that calls the xeno-canto api to get a list of recording ids for a given bird

In [ ]:
BASE_URL = 'https://xeno-canto.org/api/2/recordings?query='

last_call = None  # Used to prevent calling the API too frequently. Limit is 1 per second.

def call_api(query: dict, last_call=last_call):
    """Call the xeno-canto API with the given query parameters."""
        
    # Build the query string
    params = ['grp:"birds"']
    for key, val in query.items():
        params.append(f'{key}:"{val}"')
    param_str = ' '.join(params)
    
    # Check if cached already
    cache_str = param_str.replace(" ", "").replace('"', "").replace(":", "")
    if os.path.exists(f"./data/api_cache/{cache_str}.json"):
        with open(f"./data/api_cache/{cache_str}.json", 'r') as f:
            cached_data = json.loads(f.read())
            return {
                'numRecordings': cached_data['numRecordings'],
                'recordings': cached_data['recordings']
            }
    
    # Ensure has been longer than 1 second since last call
    if last_call is not None:
        time_since_last_call = time.time() - last_call
        if time_since_last_call < 1:
            time.sleep(1 - time_since_last_call)
    
    # Call the API
    print("Calling the api")
    res = requests.get(BASE_URL + param_str)
    json_data = res.json()
    last_call = time.time()  # Update last call time
    
    # Cache response
    CACHE_PATH = './data/api_cache'
    with open(f"{CACHE_PATH}/{cache_str}.json", 'w') as f:
        f.write(json.dumps(json_data))
    
    return {
        'numRecordings': json_data['numRecordings'],
        'recordings': json_data['recordings']
    }
    
res = call_api({
    'gen': 'fringilla',
    'ssp': 'coelebs'
})

print("Number of recordings:", res['numRecordings'])

print(json.dumps(res, indent=2))

Write a load audio function that loads a saved audio file and returns its y values and sample rate.

In [ ]:
BASE_PATH = 'data/bird_recordings'
SAMPLE_RATE = 16000

def load_audio(id: int):
    return librosa.load(f'{BASE_PATH}/{id}', sr=SAMPLE_RATE)

The following function loads the file from storage if already save, otherwise, downloads it from xeno-canto.org

In [ ]:


def download_audio(id: int):
    """Downloads the audio file with the given id from xeno-canto.org"""
    
    # Construct the file path
    file_path = f'{BASE_PATH}/{id}'
    
    # If file already exists, return it
    if os.path.exists(file_path):
        print(f"File already exists as {file_path}")
        return load_audio(id)
    
    # Make a request to download the file
    res = requests.get(f'https://xeno-canto.org/{id}/download')
    if res.status_code == 200:
        # Open the file in write-binary mode and save the content
        with open(file_path, "wb") as file:
            file.write(res.content)
        print(f"File saved successfully as {file_path}")
        return load_audio(id)
    else:
        print(f"Request failed with status code {res.status_code}")
        raise Exception(f"Failed to download file with id {id}")

Testing the above code

In [ ]:
y, sr = download_audio('622073')

ipd.display(ipd.Audio(y, rate=sr))

In [ ]:
def get_bird_recordings(bird: tuple, n = 10):
    """Get the first n recordings for the given bird species"""
    
    data = call_api({
        'gen': bird[0],
        'ssp': bird[1]
    })
    print(f"{bird[0]} {bird[1]}: {data['numRecordings']} recordings")
    
    return data['recordings']

I've selected 10 british birds that have distinct calls that will be easy to tell apart from each other. This will make training an initial model a bit easier.

In [125]:
birds = [
    ("fringilla", "coelebs"),
    ("Parus", "major"),
    ("Turdus", "merula"),
    ("Erithacus", "rubecula"),
    ("Columba", "palumbus"),
    ("Turdus", "philomelos"),
    ("Cyanistes", "caeruleus"),
    ("Cuculus", "canorus"),
    ("Troglodytes", "troglodytes"),
    ("Cuculus", "canorus")
]

rows = []

num_recordings = 50  # Number of recordings to get for each bird

for bird in birds:
    recordings = get_bird_recordings(bird)
    recordings = [
        {
            'id': recording['id'],
            'gen': recording['gen'],
            'ssp': recording['ssp']
        } for recording in recordings[:50]
    ]
    for recording in recordings:
        download_audio(recording['id'])
    rows.extend(recordings)
    
df = pd.DataFrame(rows)
    
df.sample(10)

df.to_csv('data/bird_recordings.csv', index=False)

fringilla coelebs: 1657 recordings
File already exists as data/bird_recordings/900826
File already exists as data/bird_recordings/818441
File already exists as data/bird_recordings/806671
File already exists as data/bird_recordings/793412
File already exists as data/bird_recordings/793409
File already exists as data/bird_recordings/792675
File already exists as data/bird_recordings/730707
File already exists as data/bird_recordings/721070
File already exists as data/bird_recordings/720846
File already exists as data/bird_recordings/719878
File already exists as data/bird_recordings/717157
File already exists as data/bird_recordings/717153
File already exists as data/bird_recordings/713698
File already exists as data/bird_recordings/710831
File already exists as data/bird_recordings/708258
File already exists as data/bird_recordings/696934
File already exists as data/bird_recordings/690445
File already exists as data/bird_recordings/661499
File already exists as data/bird_recordings/655